In [2]:

#Importar librerias? Pues claro que me importan!
import numpy as np
import pandas as pd
from numpy import random
from random import choices 
from scipy.stats import rv_discrete
import os 


In [2]:
#Definicion de variables 
mediaincubacion = 7
desvincubacion = 3
mediaduracion = 14
desvduracion = 5
posibilidadContagio = 0.4 #valor inicial, se puede cambiar 
cambioEstadoInicial = 15 
numpersonas = 800
personasedificio = 20
horassimulacion = 1200


In [5]:
#Definicion de una distribucion para la edad de las personas. Se basa en datos demograficos reales.
#Ahora mismo esta usando datos de España sacados del INE.
datos_edad = pd.read_csv("distribucion_edad.csv")
l=list(datos_edad.iloc[0])
l2= []
for i in range(len(l)-1):
    l2.append(int(l[i])/int(l[-1]))
#Esto es para ajustar porque parece que el numero total de personas y el numero de personas por edad no acaba de coincidir
#creo que es que corte los mayores de 100 años.
l2.append(1-sum(l2))

l3=[i for i in range(len(l2))]

#Esta es la distribución que obtenemos
dtrEdad = rv_discrete(values=(l3,l2))

#Elimino para que no queden variables sueltas
del l
del l2
del l3

In [3]:

def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.6, 0.4]
    n = choices(estado, peso_de_decision)
    return n[0]

def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [0,1]
        peso = [0.87, 0.13] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad <= 65):
        valoracion = [0,1]
        peso = [0.99, 0.01] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso) #n es un vector cuyo primer elemento es la valoracion 
    return n[0]
    

In [4]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugaroriginal, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugaroriginal
            self.lugaroriginal=lugaroriginal
            self.diasestado=0 #esto se usa?
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            self.edificiomuerte=-1
            
             
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni) + "," +str(self.edad)            

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def esta_sano(self):
            return (self.estado == "sano") or (self.estado == "inmune")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible()):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].contagiarse(posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual)
                           # print(ciudad[self.lugar][i].cambioEstado)
                            self.contadorInfecciones +=1
                            
                            
        def contagiarse(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual):
            self.estado=self.estadosposibles[1]
            self.diasestado=0
            self.cambioEstado=diaactual+(random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2)
                           
                            
        def transicionEstados(self, dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)
                if(me_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado <= dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = dia + random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2    #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado =dia+ random.randint(int(mediaduracion-desvduracion),int(mediaduracion+desvduracion)) #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 
                                     
                

In [5]:

class Simulador:
        def __init__(self, numpersonas,personasedificio):           
            self.ciudad=crearciudadv0(numpersonas,personasedificio)
            self.dia=0 #se inicializa el dia con un valor de 0 
            self.hora=0 #se inicializa las horas con un valor de 0 
            self.numpersonasinicial=numpersonas 
            self.numpersonas=numpersonas
            self.numedificios=len(self.ciudad)
            self.capacidades=np.zeros(self.numedificios)
            self.CatalogoPersonas=[]
            self.cementerio=[]
            for i in range(self.numedificios-1,-1,-1):
                self.capacidades[i]=int(len(self.ciudad[i])) #establecemos las capacidades de los edificios
                for j in range (len(self.ciudad[i])):
                    self.CatalogoPersonas.append(self.ciudad[i][j])
                    
            self.capacidades=self.capacidades.astype(int)
            print(self.capacidades)
            #for i in range(len(self.CatalogoPersonas)):
            #    print(self.CatalogoPersonas[i])
            
                
           
        def __str__(self):
            #Devuelve un string con la informacion util del simulador
            return str(self.dia) + "," + str(self.hora) + "," + str(self.numpersonas) + "," +str(self.numedificios)  
        
        def actualizar_catalogo(self):#actualizamos los datos de las personas ordenadas por id
            nuevoCatalogo=[]
            for i in range(self.numedificios):
                for j in range (len(self.ciudad[i])):
                    nuevoCatalogo.append(self.ciudad[i][j]) #primero cogemos todas las personas sin ordenar
                    self.CatalogoPersonas = sorted(nuevoCatalogo, key=lambda x: x.idpersona, reverse=False)
            
        #Hace que el simulador avance una hora , con sus consecuencias en las personas
        def pasar_tiempo(self,mediaincubacion,desvincubacion,mediaduracion,desvduracion,posibilidadContagio ):
            if ((self.hora % 8)==0): #barajar a las personas cada 8 horas( hacen shuffle)
                self.moverpersonas2()
                #print("owo")
            for j in range(len(self.ciudad)): #recorre los edificios 
                for i in range(len(self.ciudad[j])): #recorre las personas 
                    self.ciudad[j][i].infectar(posibilidadContagio,self.ciudad,mediaincubacion,desvincubacion,self.dia) #pasa a infectar 
            if (self.hora<23):
                self.hora+=1
            else:#comienza un nuevo dia
                self.hora=0
                self.dia+=1
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):                     
                        self.ciudad[j][i].transicionEstados(self.dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion)
                        
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])-1,-1,-1):   
                        if (self.ciudad[j][i].estado=="muerto"):
                            
                            self.cementerio.append(self.ciudad[j].pop(i))
                            self.numpersonas-=1
                self.cementerio=sorted(self.cementerio, key=lambda x: x.idpersona, reverse=False)
                            
        
        #Función que devuelve el total de personas sanas o que ya pasaron la enfermedad de la ciudad
        def personas_sanas(self):
            contador_sanos=0
            for j in range(len(self.ciudad)): #se recorre los edificios de la ciudad
                    for i in range(len(self.ciudad[j])): #se va recorriendo las personas de los edificios 
                        if self.ciudad[j][i].esta_sano():
                            contador_sanos+=1
            return contador_sanos
        
        def cogerpersona(self, idpersona):
            self.actualizar_catalogo()
            return self.CatalogoPersonas[idpersona]
        
        
        
        def moverpersonas(self):#Baraja a todas las personas de la ciudad indiscriminadamente
                                #Vamos a crear una nueva ciudad utilizando las personas actuales
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosSinVaciar=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            for i in range(self.numedificios-1,-1,-1):
                if (len(self.ciudad[i])==0):
                    del self.ciudad[i]
                    edificiosSinVaciar-=1
            
            for i in range(self.numedificios):
                edificionuevo = [] #El edificio actual de la ciudad nueva.
                
                for j in range(self.capacidades[i]):
                    if personasColocadas==self.numpersonas:
                        break
                    
                    if edificiosSinVaciar>1:
                        edificioescogido=random.randint(0,edificiosSinVaciar)
                    else:
                        edificioescogido=0
                    edificionuevo.append(self.ciudad[edificioescogido].pop(0))
                    edificionuevo[j].lugar=i
                    personasColocadas+=1
                    if (personasColocadas==self.numpersonas):
                        break
                    if (len(self.ciudad[edificioescogido])==0):
                        del  self.ciudad[edificioescogido]
                        edificiosSinVaciar-=1
            
                ciudadnueva.append(edificionuevo)
            self.ciudad=ciudadnueva
            
        def moverpersonas2(self):
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosAntiguos=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            opciones=[]
            for i in range(self.numedificios):
                edificionuevo=[]
                ciudadnueva.append(edificionuevo)
                opciones.append(i)
            
            personasColocadas=0
            edificiosLlenos=self.numedificios
            
            for i in range(self.numedificios):
                for j in range(len(self.ciudad[i])):
                    if (len(self.ciudad[i])==0):
                        break
                    edificioALlenar=random.choice(opciones)
                    ciudadnueva[edificioALlenar].append(self.ciudad[i].pop(0))
                    if (len(ciudadnueva[edificioALlenar])==self.capacidades[edificioALlenar]):
                        opciones.remove(edificioALlenar)
            self.ciudad=ciudadnueva
            
            
        #Función que contagia a la primera persona (la deja sintomática), se usa para crear pacientes 0
        def contagio_fijo(self, cambioEstadoInicial):
            self.ciudad[0][0].estado=self.ciudad[0][0].estadosposibles[3]
            self.ciudad[0][0].cambioEstado= cambioEstadoInicial
            
            
        def mostrarcementerio(self):#pues eso
            print(f"Total de muertos:{len(self.cementerio)}")
            for j in range(len(self.cementerio)):
                print(self.cementerio[j])

In [6]:
#Funcion que se encarga de generar la edad de manera aleatoria
#Esta funcion ya no deberia ser necesaria
def generador_edad():
        n = dtrEdad.rvs(size=1) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [7]:
#Funcion que se encarga de generar numeros aleatorios 
def generador_numeros(longitud):
    numero=0
    for i in range(0,longitud): 
        n = random.randint(0,9)
        numero=10*numero+n
    return numero


In [8]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni():
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni
    input=generador_numeros(8)
    #Se consigue que toda la secuencia de numeros sea 1 solo
    
    #Se calcula la letra que ha de ser adicionada
    valor =int(input / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = input - valor;
    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia 
    DNI=str(input)+letras[valor]
    
    return DNI

In [9]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        #print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            #print(f"contador del edificio:{contadoredificio}")
            #print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            column.append(Persona(capacidadusada,generador_edad(),edificioactual, creaDni()))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
     for j in range(len(ciudad[i])):
        print(ciudad[i][j])
    

In [10]:
MiSimulador=Simulador(numpersonas,personasedificio) #numPersonas, personasEdificio 
print(MiSimulador)
#se crea un paciente que resulta ser el 0, pasandole 15 dias como dia para el cambio de estadp 
MiSimulador.contagio_fijo(cambioEstadoInicial) 
for i in range(horassimulacion): #horas que pasa la simulacion 
    '''
    Datos inventados:
    MediaIncubación:7 dias
    DesvIncubación:3 dias
    MediaDuracion:14 dias
    DescDuracion:5 dias
    '''
    MiSimulador.pasar_tiempo(mediaincubacion,desvincubacion,mediaduracion,desvduracion,posibilidadContagio)
    if MiSimulador.hora==0:
        print("--------")
        print(" ")
        print(" ")
        print(MiSimulador.dia)
        print(MiSimulador.capacidades)
        print(MiSimulador)
        printearciudad(MiSimulador.ciudad)
        print(MiSimulador.personas_sanas())
        print(" ")
        print(" ")
        print("--------")
        print(" ")
        print(" ")
        MiSimulador.mostrarcementerio()
        print(" ")
        print(" ")
        print("--------")

[ 7  8 34 57  2 55 30 23 31 44 25 46 29 42 26 26 35 40 42 19 59 37 59 24]
0,0,800,24
Total de vivos:800
Total de vivos:800
Total de vivos:800
--------
 
 
1
[ 7  8 34 57  2 55 30 23 31 44 25 46 29 42 26 26 35 40 42 19 59 37 59 24]
1,0,800,24
8,sano,1,76530624X,9
99,sano,3,41414605T,85
340,sano,11,16151626Z,83
212,sano,7,41014451T,46
603,sano,19,73458551R,53
798,sano,23,17863808F,83
195,sano,7,2448761C,78

396,sano,13,2714666E,47
54,sano,3,33333164T,65
670,sano,20,55245670S,8
517,sano,16,83320560R,41
791,sano,23,6555760R,35
640,sano,20,37527658F,66
19,sano,2,58256723T,1
70,sano,3,2026656B,77

732,sano,22,74767542Q,65
344,sano,11,34116508X,51
41,sano,2,55825572H,40
760,sano,22,11180366C,91
444,sano,14,8316802W,85
712,sano,21,3235362K,65
669,sano,20,84437277E,107
267,sano,9,5002021G,8
314,sano,10,8134074D,15
22,sano,2,68000253V,68
368,sano,12,34723524X,10
206,sano,7,88225347S,108
243,sano,8,58670546F,10
191,sano,6,60042352R,21
631,sano,20,41784118H,85
571,sano,18,35165801C,3
426,sano,13,6

586,sano,18,61067660S,72
672,sano,20,67144434M,92
210,sano,7,633621V,2
617,sano,19,66356267W,31
688,sano,21,42861478N,55
249,sano,9,37286006Q,100
402,sano,13,60314775N,73
499,sano,16,75522288V,106
664,sano,20,28387105E,29
522,sano,17,88813331G,69
283,sano,9,26036736T,56
490,sano,16,50081101G,109
139,sano,5,18716284B,2
159,sano,5,7422340X,74
225,sano,8,14830132P,17
140,sano,5,11151362L,1
574,sano,18,61208065M,52
306,sano,10,77511674H,23
681,sano,21,1172646Z,33

328,sano,11,41426211Z,87
465,incubando,15,67833287P,66
736,sano,22,5031642R,70
252,sano,9,8675453Z,36
84,sano,3,54737377E,35
461,sano,15,81533826E,65
39,sano,2,25307720S,18
226,sano,8,48872381A,70
51,sano,3,20570737C,92
782,sano,23,37735466X,22
792,sano,23,85213048A,60
678,sano,20,47462871W,16
759,sano,22,12370856F,36
514,sano,16,11610348V,20
111,sano,5,51455052G,4
692,sano,21,37047534P,79
790,sano,23,7702770R,36
702,sano,21,54061678V,55
95,sano,3,62184416Y,52
227,sano,8,28576548Z,37
256,sano,9,45334736B,35
364,sano,12,42443514G,

242,sano,8,40521727M,2
152,sano,5,57200150R,28
718,sano,22,54667266S,26
532,sano,17,20566883F,87
426,sano,13,61257866B,71
376,sano,12,41860725N,54
770,sano,22,8542034H,40
169,sano,6,66288721F,21
142,sano,5,3604031T,105
118,sano,5,53456288X,104
575,sano,18,52317584J,59
717,sano,22,60872252S,77
465,incubando,15,67833287P,66
64,sano,3,80885143P,38
314,sano,10,8134074D,15

456,sano,14,66236837B,26
485,sano,16,58364804G,37

562,sano,18,15872480L,91
288,sano,9,6426673J,75
177,sano,6,86410884E,44
494,sano,16,82184732M,84
478,sano,15,24656736T,59
385,sano,12,40122470M,33
112,sano,5,40124024H,24
612,sano,19,78342744Y,29
327,sano,11,67668021C,70
126,sano,5,71217013C,3
342,sano,11,43711226W,70
441,sano,14,24781086N,56
690,sano,21,37235880F,99
57,sano,3,35711865H,13
287,sano,9,47252266P,52
200,sano,7,71758575R,3
156,sano,5,60676673M,37
487,sano,16,13567146K,82
476,sano,15,81060865X,84
454,sano,14,70563284C,67
304,sano,10,83025268Y,68
715,sano,21,57144740K,98
150,sano,5,88202047Z,49
573,sano,18,658

76,sano,3,15304457G,62
688,sano,21,42861478N,55
168,sano,6,33821783F,96
524,sano,17,43421240T,82
551,sano,17,13632537T,86
500,sano,16,624482D,79
250,sano,9,3351635Y,46
26,sano,2,72170843V,35
283,sano,9,26036736T,56
512,sano,16,75105125M,90
719,sano,22,72755308Y,30
720,sano,22,25811265K,107
675,sano,20,82142540H,3
91,sano,3,83726675M,102
170,sano,6,50667831G,73
643,sano,20,48166148P,68
799,sano,23,82575530X,76
522,sano,17,88813331G,69
103,sano,3,43705555N,94

625,sano,20,41515185R,101
619,sano,19,22074864Q,24
192,sano,6,33258553R,9
404,sano,13,72321443J,53
107,sano,4,8647306L,26
179,sano,6,52053517D,87
106,sano,4,26310305F,45
83,sano,3,71762437E,83
492,sano,16,78458611E,22
199,sano,7,17632223D,47
239,sano,8,88050518D,56
529,sano,17,61105020T,70
560,sano,18,87225484P,7
293,sano,10,5548602J,16
351,sano,11,24185208H,65
299,sano,10,76042764A,82
738,sano,22,82535068M,102
88,sano,3,31037240M,54
95,sano,3,62184416Y,52

763,sano,22,26723568Y,38
270,sano,9,85231080A,104
637,sano,20,28034082A,60


404,sano,13,72321443J,53
398,sano,13,42528755F,103
744,sano,22,35444651H,75
378,sano,12,35262317M,100
438,sano,14,51132842R,1
149,sano,5,87725182P,97
538,sano,17,45836513C,92
241,sano,8,71748562Q,96
729,sano,22,44704886Q,71
54,sano,3,33333164T,65
416,sano,13,28331387X,22
145,sano,5,57844024J,65
524,sano,17,43421240T,82
323,sano,11,37421421F,59
91,sano,3,83726675M,102
350,sano,11,27283373Z,40
396,sano,13,2714666E,47
727,sano,22,87418784Q,70
648,sano,20,83626661H,63
753,sano,22,53451681A,54
413,sano,13,67426287Q,101
291,sano,10,24670880E,62
361,sano,11,72377615L,101
461,sano,15,81533826E,65
682,sano,21,82734538L,37
604,sano,19,41056208N,3
238,sano,8,40457312J,23
175,sano,6,53015428Z,55
263,sano,9,74241483J,87
796,sano,23,80578453T,33
646,sano,20,51804518P,59
407,sano,13,48081741B,58
157,sano,5,46503858C,12
10,sano,1,84781288E,78
65,sano,3,51747665B,14

740,sano,22,51306538R,29
349,sano,11,85443441M,51
605,sano,19,4605540C,27
534,sano,17,11004047L,77
163,sano,6,5805530P,2
208,sano,7,68286

...

682,inmune,21,82734538L,37
310,inmune,10,10422267R,108
760,inmune,22,11180366C,91
291,inmune,10,24670880E,62
313,inmune,10,5077360H,3
249,inmune,9,37286006Q,100
532,inmune,17,20566883F,87
670,inmune,20,55245670S,8
113,inmune,5,22123455P,100
797,inmune,23,20746365C,43
187,inmune,6,84483407Z,33
526,inmune,17,53381038Q,30
477,inmune,15,3235624F,53

607,inmune,19,84746250J,107
98,inmune,3,3565306F,80
177,inmune,6,86410884E,44
499,inmune,16,75522288V,106
88,inmune,3,31037240M,54
579,inmune,18,16586621X,89
1,inmune,0,50160011R,4
629,inmune,20,4764231B,74
380,inmune,12,72470108Y,32
358,inmune,11,33077340M,23
675,inmune,20,82142540H,3
698,inmune,21,66524545N,71
89,inmune,3,86030854K,109
456,inmune,14,66236837B,26
115,inmune,5,78125530G,3
648,inmune,20,83626661H,63
398,inmune,13,42528755F,103
619,inmune,19,22074864Q,24
627,inmune,20,27868035Q,7
186,inmune,6,46524077E,79
34,inmune,2,17735572L,98
425,inmune,13,66454234N,48
700,inmune,21,78101844P,27
365,inmune,12,68386413F,7
326,inmune,11,7083538

28,inmune,2,42587584W,13
137,inmune,5,75812234W,105
214,inmune,7,75304148D,40
39,inmune,2,25307720S,18
313,inmune,10,5077360H,3
781,inmune,23,47788014V,58

656,inmune,20,42148522X,109
699,inmune,21,70830840V,104
558,inmune,17,5480211R,1
752,inmune,22,12223866X,31
160,inmune,5,34640048R,35
75,inmune,3,67514254P,97
293,inmune,10,5548602J,16
405,inmune,13,28202185E,75
343,inmune,11,87754888K,12
505,inmune,16,63730081W,36
622,inmune,20,30085330L,0
494,inmune,16,82184732M,84
224,inmune,8,62806253S,3
44,inmune,2,65608264M,75
411,inmune,13,80705542Z,2
567,inmune,18,54726344Y,96
650,inmune,20,84565527R,87
580,inmune,18,61087185J,3
633,inmune,20,83270536W,99
282,inmune,9,13243400T,54
110,inmune,5,81034501G,8
385,inmune,12,40122470M,33
617,inmune,19,66356267W,31
573,inmune,18,65820060V,64
311,inmune,10,3800046D,100
517,inmune,16,83320560R,41
89,inmune,3,86030854K,109
640,inmune,20,37527658F,66
11,inmune,1,34837711W,34
470,inmune,15,503605C,53
782,inmune,23,37735466X,22
309,inmune,10,82458843W,43

571,inmune,18,35165801C,3
401,inmune,13,76656020X,29
366,inmune,12,66057584C,74
724,inmune,22,54671717G,0
781,inmune,23,47788014V,58
757,inmune,22,61866667W,30
692,inmune,21,37047534P,79
752,inmune,22,12223866X,31
122,inmune,5,40720386J,86
449,inmune,14,57666825Y,93
233,inmune,8,71171301D,67
249,inmune,9,37286006Q,100
664,inmune,20,28387105E,29
481,inmune,15,48452484V,2
628,inmune,20,82885713S,5
468,inmune,15,33575178P,77

667
 
 
--------
 
 
Total de muertos:133
6,muerto,0,18318322H,73
10,muerto,1,84781288E,78
15,muerto,2,36121484F,104
29,muerto,2,62033464A,36
30,muerto,2,56473453N,101
41,muerto,2,55825572H,40
47,muerto,2,12176403L,86
53,muerto,3,42580263H,74
59,muerto,3,26024564H,66
62,muerto,3,37861653L,101
66,muerto,3,75208683V,97
67,muerto,3,22001017E,86
68,muerto,3,38858567K,89
69,muerto,3,63350118E,109
70,muerto,3,2026656B,77
74,muerto,3,73464544Z,108
78,muerto,3,16172854J,97
80,muerto,3,73625082N,72
81,muerto,3,63507548V,88
87,muerto,3,10254348M,69
91,muerto,3,83726675M,102
99

474,inmune,15,10310870Q,17
587,inmune,18,72565417A,69
758,inmune,22,14343665Z,28
732,inmune,22,74767542Q,65
714,inmune,21,36200883X,43
333,inmune,11,37083732G,57
571,inmune,18,35165801C,3

300,inmune,10,1330851W,77
511,inmune,16,7737020G,51
250,inmune,9,3351635Y,46
589,inmune,18,86663415J,59
186,inmune,6,46524077E,79
397,inmune,13,88641310T,36
687,inmune,21,56183254G,97
86,inmune,3,68785164F,49

197,inmune,7,87550250Z,24
467,inmune,15,23268183A,24
57,inmune,3,35711865H,13
683,inmune,21,11670730R,22
700,inmune,21,78101844P,27
320,inmune,11,36360533V,89
237,inmune,8,23126453E,50
590,inmune,18,4453306T,1
442,inmune,14,63223150Z,24
743,inmune,22,2656474C,18
702,inmune,21,54061678V,55
670,inmune,20,55245670S,8
251,inmune,9,33364437Q,12
358,inmune,11,33077340M,23
156,inmune,5,60676673M,37
609,inmune,19,74768710B,99
541,inmune,17,23044423X,27
299,inmune,10,76042764A,82
349,inmune,11,85443441M,51
9,inmune,1,34803716R,54
181,inmune,6,57634810F,6
575,inmune,18,52317584J,59
450,inmune,14,58827017

458,inmune,14,12227122T,71
618,inmune,19,15135202Y,30
401,inmune,13,76656020X,29
554,inmune,17,67115534Q,56
696,inmune,21,46557043Y,26
12,inmune,1,86163243E,56
704,inmune,21,56764584D,3
637,inmune,20,28034082A,60
501,inmune,16,12071528R,18
309,inmune,10,82458843W,43
520,inmune,17,85354043P,1
642,inmune,20,44021524F,72
24,inmune,2,5385786Z,3
230,inmune,8,81876522H,53
253,inmune,9,58875344J,5
299,inmune,10,76042764A,82
294,inmune,10,84307236T,68

8,inmune,1,76530624X,9
176,inmune,6,24826480G,46
455,inmune,14,15110137B,36
338,inmune,11,56483744E,31
524,inmune,17,43421240T,82
384,inmune,12,72341173D,53
575,inmune,18,52317584J,59
514,inmune,16,11610348V,20
585,inmune,18,74362015W,17
98,inmune,3,3565306F,80
156,inmune,5,60676673M,37
194,inmune,7,5882521H,11
614,inmune,19,62825443T,30
794,inmune,23,47812187V,33
495,inmune,16,255636Z,15
162,inmune,5,33714033N,57
391,inmune,13,36815400J,108
284,inmune,9,27541308G,68
589,inmune,18,86663415J,59
671,inmune,20,13067242E,64
459,inmune,15,26014820A,4

In [25]:
printearEdificio(MiSimulador.ciudad,0)

68,inmune,3,56711266M,22


In [26]:
printearciudad(MiSimulador.ciudad)

68,inmune,3,56711266M,22

22,inmune,1,44622371W,57
18,inmune,1,81356210N,8
76,inmune,4,88524085Y,59
23,inmune,1,86670864X,4
32,inmune,1,42454783A,11
20,inmune,1,53428358W,78
3,inmune,1,27102032M,34
83,inmune,4,77414261X,56
72,inmune,4,25210143G,20
30,inmune,1,58846701M,39
48,inmune,2,1162457Z,63
88,inmune,4,33846622Y,15
57,inmune,2,71436206T,22
11,inmune,1,48667561K,107
93,inmune,4,68352384H,80
4,inmune,1,11776370W,28
43,inmune,2,4277068B,35
98,inmune,5,46154545P,101
97,inmune,5,60588057P,32
19,inmune,1,15120414F,10
65,inmune,3,34372778Z,100

8,inmune,1,53054467E,46
36,inmune,2,26306750V,18
89,inmune,4,34758580Z,78
80,inmune,4,72888650V,86
67,inmune,3,86035467B,91
53,inmune,2,8833281Q,63
15,inmune,1,2846257F,85
50,inmune,2,13316802D,39
84,inmune,4,60008807J,36
2,inmune,1,43602763F,64
29,inmune,1,80645358K,47
55,inmune,2,41147331D,35
64,inmune,3,58076113D,101
44,inmune,2,3608884T,92
35,inmune,1,41825024F,92
59,inmune,2,30124532Y,9
78,inmune,4,18158546T,9
25,inmune,1,5302348C,34
79,inmun

In [27]:
MiSimulador.actualizar_catalogo()
for i in range(len(MiSimulador.CatalogoPersonas)):
                print(MiSimulador.CatalogoPersonas[i])
            

0,inmune,0,11243867H,25
1,inmune,1,56224405P,70
2,inmune,1,43602763F,64
3,inmune,1,27102032M,34
4,inmune,1,11776370W,28
5,inmune,1,87008361M,2
7,inmune,1,55601061X,7
8,inmune,1,53054467E,46
10,inmune,1,86644448K,22
11,inmune,1,48667561K,107
12,inmune,1,2470853D,48
13,inmune,1,24375770W,21
14,inmune,1,75147732Q,71
15,inmune,1,2846257F,85
18,inmune,1,81356210N,8
19,inmune,1,15120414F,10
20,inmune,1,53428358W,78
22,inmune,1,44622371W,57
23,inmune,1,86670864X,4
25,inmune,1,5302348C,34
28,inmune,1,82581208F,67
29,inmune,1,80645358K,47
30,inmune,1,58846701M,39
32,inmune,1,42454783A,11
33,inmune,1,76654370Q,35
34,inmune,1,63281802Q,48
35,inmune,1,41825024F,92
36,inmune,2,26306750V,18
37,inmune,2,50446516H,104
39,inmune,2,43800534R,6
41,inmune,2,40557607M,49
42,inmune,2,76482600X,35
43,inmune,2,4277068B,35
44,inmune,2,3608884T,92
45,inmune,2,47076434B,42
47,inmune,2,61775203D,34
48,inmune,2,1162457Z,63
49,inmune,2,11447256H,44
50,inmune,2,13316802D,39
51,inmune,2,16025612V,56
53,inmune,2,88332

In [28]:
personitaputa=MiSimulador.cogerpersona(0)
print(personitaputa.contadorInfecciones)

4


In [167]:
MiSimulador=Simulador(100,10) 
print(MiSimulador.capacidades)
print(len(MiSimulador.ciudad))

[23  1 29  7 22 16  2]
0,sano,0,87204200E,7
1,sano,0,81351675P,65
2,sano,0,7541462S,84
3,sano,0,2734837E,49
4,sano,0,67118311X,62
5,sano,0,63653820D,41
6,sano,0,56840305Z,104
7,sano,0,71438685H,88
8,sano,0,37235031D,2
9,sano,0,24211256F,28
10,sano,0,76816004Y,83
11,sano,0,25172814G,49
12,sano,0,75582883F,33
13,sano,0,65767330A,93
14,sano,0,22875655Q,88
15,sano,0,11256111A,85
16,sano,0,87500174D,37
17,sano,0,23444251Y,63
18,sano,0,50162442V,58
19,sano,0,52664477C,99
20,sano,0,10176827V,22
21,sano,0,57341107Z,72
22,sano,0,32625576F,67
23,sano,1,76036012J,70
24,sano,2,26170230W,84
25,sano,2,30847235A,11
26,sano,2,78242507A,47
27,sano,2,12318218Q,13
28,sano,2,3440412A,53
29,sano,2,32441324P,31
30,sano,2,26126301A,94
31,sano,2,28610526K,51
32,sano,2,27767452N,105
33,sano,2,50417272F,78
34,sano,2,2504088D,32
35,sano,2,57177760J,40
36,sano,2,31066118H,88
37,sano,2,76265586R,79
38,sano,2,68173116N,58
39,sano,2,35280872E,52
40,sano,2,34084064L,36
41,sano,2,6287818D,23
42,sano,2,11550463R,81
43,